Conda pour le venv

In [1]:
# 1.créer un nouvel env conda à partir du terminal
# conda create --name asr_rag python=3.10

# 2. activer cet environnement
# conda activate asr_rag

# 3. installer les dépendences
# pip install -r requirements.txt

# 4. installer ollama: https://www.ollama.com/download


# 5. installer le modème d'OllamaEmbeddings
# ollama pull embeddinggemma

# 6. créer une clé api sur openrouter pour utiliser des llm gratuitement

# 7. mettre votre clé dans le fichier .env (racine du repertoire courant)

UV pour le venv (plus rapide et efficace dans la résolution des conflits de versions)

# -------------------------------------------------
1. Installer uv (si ce n’est pas déjà fait)
```bash
curl -LsSf https://uv.run
```

# -------------------------------------------------
2. Aller dans la racine du repo (ici MVP) et initialiser le projet uv
```bash
uv init
```

# -------------------------------------------------
3. Créer l’environnement virtuel
```bash
uv venv --python 3.11
```

# -------------------------------------------------
4. activer cet environnement
> Linux / macOS
> ```bash
> source .venv/bin/activate
> ```

# -------------------------------------------------
5. aller avec le terminal dans le dossier RAG installer les dépendences
```bash
uv pip install -r requirements.txt
```

# -------------------------------------------------
6. installer ollama: https://www.ollama.com/download

# -------------------------------------------------
7. installer le modème d'OllamaEmbeddings
```bash
ollama pull embeddinggemma
```

# -------------------------------------------------
8. créer une clé api sur openrouter pour utiliser des llm gratuitement

9. mettre votre clé dans le fichier .env (racine du repertoire courant)


In [2]:
from openai import OpenAI, AsyncOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.schema.document import Document
from langchain_unstructured import UnstructuredLoader
from pathrag_retriever import create_graphdb, load_existing_graphdb, load_knowledgeGraph_vis
import time
import os
from dotenv import load_dotenv

load_dotenv(".env")

OPENROUTER_API_KEY=os.getenv("OPENROUTER_API_KEY")


/home/chougar/miniconda3/envs/pathrag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Création ou chargement d'un graphe existant:
> Lorsque vous executer la cellule ci dessous, vous avez 2 options proposées:
> 1. Créer un nouveau graphe
> 2. Charger un graphe existant
> 
> Un prompt de sélection s'affichera en haut du notebook<br>
> Saisir l'action désirée dans le , suivre instructions

> Si vous voulez charger un graphe déjà crée et que vous ne savez plus son nom, retrouver le dans le fichier `graphrag_hashes.json` (dans la racine du dossier), attribut `Nom du doc`

> Si vous voulez modifier le LLM utilisé pour la création du graphe ou sa lecture, allez dans `pathrag_retriever.py`, sous la ligne 36( Choix du LLM OpenRouter), et prenez un modèle valide sur openrouter (attention à prendre un modèle qui supporte les `structured_outputs`, sur le site openrouter, à filtrer sur le paneau à gauche dans la liste des `Supported parameters`)

#### 1. Créer un graphe, ou charger en un déjà crée
Ci dessous un print des noms de docs disponibles

In [3]:
import pandas as pd

graph_names=pd.read_json("graphrag_hashes.json")

graph_names=graph_names.loc["Nom du doc", :].values.tolist()
for n in graph_names:
    print(n)

IA conscience
L-IA-notre-deuxieme-conscience


In [22]:
# 1.
import nest_asyncio

nest_asyncio.apply()

#==== params de base====

# Obligatoire: nom des sources
filename="audio-text.txt"
doc_name_graph="L-IA-notre-deuxieme-conscience"

# Optionnels:
# 1. modèle pour création du graph
# param 'OPENROUTER_MODEL_graph_creation', par défaut "deepseek/deepseek-chat-v3-0324"
# passer dans la fonction 'create_graphdb' un autre modèle si vous le souhaitez

# 2. modèle pour lecture du graph (questions/réponses)
# param 'OPENROUTER_MODEL_graph_read', par défaut "deepseek/deepseek-chat-v3-0324"
# passer dans la fonction 'load_existing_graphdb' un autre modèle si vous le souhaitez

#=======================

loader = UnstructuredLoader(filename)

txt_docs = loader.load()
text=""
for doc in txt_docs:
    text+=doc.page_content


graphrag_action=input("Saisir 'C' pour créer un nouveau graphe, 'L' pour charger un graphe existant")

# créer un nouveau graphe
messages=None

if graphrag_action=='C':
    print(f"Le nom de votre graphe est {doc_name_graph}")
    messages= create_graphdb(
        text=text, 
        doc_name=doc_name_graph, # il faut donner un nom unique permettant d'identifier et charger le graph les prochaines fois
    )
# charger un graphe existant
elif graphrag_action=='L':    
    print(f"Le nom de votre graphe est {doc_name_graph}")

    messages=load_existing_graphdb(
        doc_name_graph, 
        OPENROUTER_MODEL_graph_read="mistralai/mistral-large-2512"
    )
else:
    print('Option invalide')



if messages:
    pipeline_args={}
    for feedback in messages:
        if isinstance(feedback, str):
            print(feedback)
        elif isinstance(feedback, dict):
            pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]=feedback["pipeline_args"]
            
print("Confirmation LLM read:", pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]["llm_graph_QA"])

INFO:PathRAG:Logger initialized for working directory: /home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/9fa530c1fd2fc3cee4831f8edcd4822b9353bd24d1dbabef4a12af5a9f4501f5
INFO:PathRAG:Load KV llm_response_cache with 0 data
INFO:PathRAG:Load KV full_docs with 1 data
INFO:PathRAG:Load KV text_chunks with 11 data
INFO:PathRAG:Loaded graph from /home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/9fa530c1fd2fc3cee4831f8edcd4822b9353bd24d1dbabef4a12af5a9f4501f5/graph_chunk_entity_relation.graphml with 84 nodes, 56 edges
INFO:nano-vectordb:Load (82, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Documents/GitHub/experiments/associatif/IA audiovisuel/RAG/storage/graph_stores/9fa530c1fd2fc3cee4831f8edcd4822b9353bd24d1dbabef4a12af5a9f4501f5/vdb_entities.json'} 82 data
INFO:nano-vectordb:Load (56, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768,

Le nom de votre graphe est L-IA-notre-deuxieme-conscience

        ----------------
        #### Graph RAG retriever
        Chargement de la base Graph RAG
    
**✅ Graph RAG chargé**
Confirmation LLM read: mistralai/mistral-large-2512


#### 2. Poser vos questions au graphrag

In [ ]:
from PathRAG import QueryParam
import asyncio

def stream_pathRAG_response(stream_resp):
    async def stream_response():        
        # Process the async generator
        async for chunk in stream_resp:
            print(chunk or "", end="")


    # Run in Streamlit's existing event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(stream_response())

# question="résume ce texte dans sa langue source"
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"

resp=pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]["rag"].query(
    query= question, 
    param=QueryParam(mode="hybrid", stream=True,)
)

stream_pathRAG_response(resp)

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:PathRAG:kw_prompt result:
INFO:PathRAG:```json
{
  "high_level_keywords": [
    "Thèmes principaux",
    "Analyse de texte",
    "Compréhension de contenu",
    "Interprétation littéraire",
    "Questions pertinentes"
  ],
  "low_level_keywords": [
    "Idées centrales",
    "Sujets abordés",
    "Problématiques soulevées",
    "Axes de réflexion",
    "Exemples de questions",
    "Structure du texte",
    "Messages clés"
  ]
}
```
INFO:PathRAG:Local query uses 40 entites, 15 relations, 4 text units
INFO:PathRAG:Global query uses 55 entites, 40 relations, 3 text units
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


response all ready
Ce texte, issu d’une discussion animée sur *France Culture*, explore les enjeux philosophiques, techniques et sociétaux de l’intelligence artificielle (IA) à travers les échanges entre plusieurs intervenants, dont **Nathan Devers** (animateur), **Laurence Devillers** (spécialiste de l’IA et de la manipulation), **Valentin Husson** (philosophe) et **Daniel Andler** (mathématicien et philosophe). Les thèmes abordés sont riches et multidimensionnels, mêlant analyses historiques, critiques technologiques et réflexions sur l’avenir de l’humanité. Voici une synthèse des principaux thèmes et des questions qu’ils soulèvent.

---

### **1. Les limites et illusions de l’intelligence artificielle**
Un des fils conducteurs du débat est la **démystification de l’IA**, souvent présentée comme une révolution capable de rivaliser avec l’intelligence humaine. Les intervenants insistent sur ses **limites fondamentales** :
- **Absence de conscience et d’intention** : Laurence Devillers

============================
### RAG vectoriel
1. Embedding du document -> renseigner le nom de votre fichier dans `filename` et le nom de votre DB dans `doc_name_hybrid`
2. Setup du retriever / reranker / llm

In [28]:
#1
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.retrievers import TFIDFRetriever
from langchain_unstructured import UnstructuredLoader

# chargement et fragmentation du doc
## Nom du doc à traiter
filename="audio-text.txt"

## Nom pour la base vectorielle
doc_name_hybrid="L-IA-notre-deuxieme-conscience_sample" # nom de doc significatif



#========= choix du modèle d'embedding
"""
    Le modèle choisi impacte la qualité du retriever, mais aussi le temps de traitement
    Si le déploiement est prévu sur une VM limitée, un modèle plus petit est nécessaire
    Explorer les comparatifs: https://huggingface.co/spaces/mteb/leaderboard

"""
# Utiliser OllamaEmbeddings avec le modèle local "embeddinggemma"
embeddings = OllamaEmbeddings(model="embeddinggemma")


# loader = UnstructuredFileLoader(filename)
loader = UnstructuredLoader(filename)

txt_doc = loader.load()
print(f"Loaded {len(txt_doc)} documents from {filename}")


#======== choix des paramètres de fragmentation
"""
    la taille du chunck_size est très important dans l'accès à une info précise
    une plus petite taille permet de cibler de courts passages contenant l'info nécessaire à des réponses précises:
        * lieu du projet
        * dates du projet
        * budget ...    
    l'envoi de passages plus courts au llm évite une dispertion de son attention
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docs = text_splitter.split_documents(txt_doc)

# Filter out complex metadata (e.g., lists, dicts)
docs = [Document(doc.page_content) for doc in docs]


# Conversion des docs en embeddings 
chroma_db = Chroma.from_documents(
    docs,
    embedding=embeddings,
    persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
    collection_name=doc_name_hybrid.replace(" ","_")
)

retriever=chroma_db.as_retriever()





# ...existing code...
all_docs = chroma_db.get()
print("Nb de chuncks:", len(all_docs['documents']))  # This will print the total number of docs stored
# ...existing code...

Loaded 1 documents from audio-text.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Nb de chuncks: 51


In [29]:
for c in all_docs["documents"][:4]:
    print(c, "\n=============")

[Nathan Devert]: France Culture. [Nathan Devert]: France Culture, sans préjuger. Nathan Devert. [Nathan Devert]: Comment expliquer les progrès phénoménaux que semble avoir accompli l'intelligence artificielle au cours de ces dernières années ? Depuis l'apparition de ChatGPT en novembre 2022, rapidement suivi par d'autres agents conversationnels, cette révolution technologique aux multiples aspects, paraît désormais capable d'exécuter de nombreuses tâches intellectuelles sur lesquelles l'esprit humain pensait jusqu'alors exercer un monopole. Écrire des articles, synthétiser des documents, traiter des données dans n'importe quel domaine, diagnostiquer une maladie, rédiger une dissertation, ou pourquoi pas, un scénario de film. Non contente de révolutionner le monde du travail, ses prouesses stupéfiantes de la technique soulèvent une interrogation majeure dans le domaine de la philosophie de l'esprit. Faut-il en déduire que l'intelligence, n'étant pas le propre d'un cerveau biologique et 

In [ ]:
#2
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.schema.document import Document
from openai import OpenAI, AsyncOpenAI
import asyncio
import json
import re
import os


class RAG_hybrid():
    def __init__(self, model):
        self.model=model
        self.retrieved_docs=[]
        self.semantic_retriever_topK=10
        self.sparse_retriever_topK=10
        self.history=[]
        self.llm_client = AsyncOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=OPENROUTER_API_KEY,
        )
        self.reranker_llm="mistralai/mistral-small-3.1-24b-instruct"
        self.doc_name_hybrid=doc_name_hybrid
        self.reranker_score_thresh=5
        self.reranked_doc=[]

    def semanticRetriever(self):
        # 1. Semantic Retriever (Chroma + OllamaEmbeddings)
        embeddings = OllamaEmbeddings(model="embeddinggemma")
        if self.doc_name_hybrid == 'None':
            return "Error: fournir le nom du document"
        
        chroma_db = Chroma(
            persist_directory=f'./storage/vector_scores/{self.doc_name_hybrid.replace(" ","_")}',
            collection_name=self.doc_name_hybrid.replace(" ","_"),
            embedding_function=embeddings
        )

        semantic_retriever=chroma_db.as_retriever(search_type="mmr", k=self.semantic_retriever_topK)

        self.chroma_db=chroma_db
        self.semantic_retriever=semantic_retriever

        return "Success: ChromaDB setup avec succes"
    
    def sparseRetriever(self):
        # 2. Sparse Retriever (TF-IDF)

        # Récupérer TOUS les documents depuis Chroma
        all_data = self.chroma_db.get(include=["documents", "metadatas"])

        # Convertir en liste de `Document` objects pour LangChain
        docs = [
            Document(page_content=text, metadata=meta or {})  # <-- Si meta est None, on met {}
            for text, meta in zip(all_data["documents"], all_data["metadatas"])
        ]

        # Créer le retriever TF-IDF
        sparse_retriever = TFIDFRetriever.from_documents(
            documents=docs,
            k=self.sparse_retriever_topK,
            tfidf_params={"min_df": 1, "ngram_range": (1, 2)}
        )

        self.sparse_retriever= sparse_retriever
    
    def ensembleRetriever(self):
        # 3. Ensemble Retriever (Semantic + Sparse)
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.semantic_retriever, self.sparse_retriever],
            weights=[0.5, 0.5]
        )

        self.ensemble_retriever=ensemble_retriever

    async def reranker(self, results, query):


        async def llm_eval(doc, query):
            system_prompt="""
                You're an expert assistant in reranking documents against a question.
                Your role is to compare the question with a document and give a score from 0 to 10, where:
                0=document out of context, unable to answer the question
                10=highly relevant document, able to answer the question
                                
                The expected final output is the score in json format
                Example:
                ```json{"score": 5}```
                
                Always end your answer with this format                
            """            
            response = await self.llm_client.chat.completions.create(
                model=self.reranker_llm,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"La question est: {query}\n Le document à évaluer est le suivant\n: {doc}" }
                ],
                temperature=0,
            )
            # Post-process to extract only the JSON part if extra text is present
            content = response.choices[0].message.content
            # Try to extract the JSON block if the model adds extra text
            match = re.search(r"\{.*?\}", content, re.DOTALL)
            if match:
                content = match.group(0)

            # extract score
            score=None
            try:
                score=content.replace("```json", "").replace("```", "")
                
                score= json.loads(score)
                score=score["score"]
            except Exception as e:
                print(e)                
            
            return {"content": doc, "score": score}


        tasks=[llm_eval(doc.page_content, query) for doc in results]
        scored_docs= await asyncio.gather(*tasks)
        i=1

        for doc in scored_docs:
          
            print(f'chunk {i} score: {doc["score"]}')
            i+=1

        filtred_docs=[d for d in scored_docs if int(d["score"])>=self.reranker_score_thresh]
        # print(f"scored docs; \n{scored_docs}")
        self.reranked_doc=filtred_docs

        return filtred_docs

    async def ask_llm(self, query):
        # 5. Final processing step with an LLM (e.g., OpenAI via OpenRouter)

        # init retrievers
        status=self.semanticRetriever()
        if "Error" in status:
            return status
        
        self.sparseRetriever()
        self.ensembleRetriever()

        # retrieve relevant docs
        results = self.ensemble_retriever.get_relevant_documents(query)
        print(f"Nb of retrieved docs: {len(results)}")

        # rerank
        scored_results=await self.reranker(results, query)
        
        # Concatenate retrieved documents for context
        context = "\n".join([f"Fragment: \n{doc['content']}\n" for doc in scored_results])

        print(f"Context lenght: {len(context.split(' '))} words")
        llm_prompt = f"""
            Answer the question based **only** on the provided context.  

            - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
            - If the context lacks relevant information, respond with: "I don't know."  

            ### **Context:**  
            {context}  

            ### **Question:**  
            {query}  

            ### **Answer:**  
            Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
        """

        llm_completion = await self.llm_client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in document Q/A and document synthesis"},
                {"role": "user", "content": llm_prompt}
            ],
            temperature=0.2,
            stream=True
        )

        final_answer = ""
        print("Réponse:\n=========")
        async for chunk in llm_completion:
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                final_answer += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="", flush=True)
        
        self.history+=[
            {"role": "user", 'content': query},
            {"role": "assistant", "content": final_answer}
        ]
        
        return final_answer



In [32]:
rag_hybrid=RAG_hybrid(model="mistralai/mistral-small-3.2-24b-instruct")

# 4. Ask a question
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"
results = await rag_hybrid.ask_llm(question)

Nb of retrieved docs: 8


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


chunk 1 score: 4
chunk 2 score: 9
chunk 3 score: 4
chunk 4 score: 9
chunk 5 score: 2
chunk 6 score: 3
chunk 7 score: 2
chunk 8 score: 7
Context lenght: 504 words


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Réponse:
### **Answer:**

#### **Principaux thèmes du texte:**

1. **Créativité et Intelligence Artificielle (IA):**
   - Le texte explore la capacité de l'IA à créer des métaphores et des analogies, en soulignant que cela relève davantage de l'optimisation mathématique que de la véritable créativité.
   - Il est mentionné que l'humain apporte une touche unique de créativité, de vision métaphorique et d'inconscient, qui trouble et enrichit les productions de l'IA.

2. **Philosophie de l'esprit et Intelligence:**
   - Le texte aborde la question de savoir si l'intelligence peut être implémentée dans des machines, et si l'IA pourrait un jour remplacer la conscience humaine.
   - Il pose également la question inverse : comment la machine peut-elle imiter les œuvres de l'esprit humain, et ce que cela révèle sur la mécanisabilité de la vie mentale.

3. **Mécanique et Créativité Humaine:**
   - Il est suggéré que la créativité humaine pourrait être vue comme une addition de méthodes, d'habit

==============================

### Evaluation des réponses
#### <u>Le cadre général:</u>
**1. Construire les réponses de référence:**
* Utiliser la question cadre "Quels sont les principaux thèmes de ce texte ?" 
* Charger le transcript du podcast et produire la réponse avec les modèles suivant:
    * Dans le chat openrouter ou par API:
        * openai/gpt-5.2
        * google/gemini-3-pro-preview
        * anthropic/claude-sonnet-4.5
    * Dans NotebookLM (solution spécialisée dans l'analyse des documents)
    
**Important**: relire et comparer les réponses avec votre compréhension du podcast, corriger et valider

**2. Utiliser les réponses de référence pour évaluer les RAGs:**

Pour la question cadre ci dessus, le graphrag est plus apte que le vectoriel

Il existe 2 paramètres techniques principaux qui influencent nettement la qualité des réponses:
* Le modèle d'embedding: 
    * Utilisé lors de la création du graphe dans la structuration des noeuds et leur connexion (par rapprochement cosine sim)
    * Utilisé lors de la lecture du graphe pour répondre à une question, qui mets en relation avec la question avec les noeuds les plus pertinent (cosine sim) pour sélectionner un/des point(s) de démarrage pour l'exploration du graphe
* Le LLM:
    * Utilisé lors de la création du graphe pour la reconnaissance des entités, leur définition a rôles, leurs catégories, les relations entre les entités, la normalisation, déduplication et unification des entités et des catégories ...
    * Utilisé lors de la génération de la réponse, en recevant en entrée les pertinentes parties du graphe comme contexte

#### Un tableau de combinaisons pour l'évaluation

Exemple de grille systématique pour tester les combinaisons de modèles d'embedding et de LLM :

## **Paramètres de Test**

### **Modèles d'Embedding (Colonnes)**
1. **Embedding bon marché**:
* gemma embeddign 300m, ...
2. **Embeddings premium**:
* gemini embedding (via openrouter)
* openai text-embedding-3-large


### **Modèles LLM (Lignes)**
**Pour la création de graphe:**
#### Modèles `performants` et bon marché, éligible pour création de graphe:
* deepseek/deepseek-chat-v3-0324: $0.20/M input tokens | $0.88/M output tokens
* deepseek/deepseek-v3.1-terminus: $0.21/M input tokens | $0.79/M output tokens
* prime-intellect/intellect-3: $0.20/M input tokens | $1.10/M output tokens
* openai/gpt-5-mini: $0,25/M input tokens | $2/M output tokens
* mistralai/mistral-large-2512: $0.50/M input tokens | $1.50/M output tokens
* mistralai/mistral-medium-3.1; $0.40/M input tokens | $2/M output tokens

#### Modèles `frontière`, meilleure qualité attendue, mais prix élevé pour modèles US:
<div class="alert" style="color: red">Attention au prix de l'output variable</div>

* google/gemini-2.5-pro: Starting at $1.25/M input tokens | Starting at $10/M output tokens
* openai/gpt-5.1: $1.25/M input tokens | $10/M output tokens
* deepseek/deepseek-v3.2: $0.28/M input tokens | $0.42/M output tokens
* moonshotai/kimi-k2-thinking: $0.45/M input tokens | $2.35/M output tokens
* z-ai/glm-4.6; $0.39/M input tokens | $1.90/M output tokens

**Pour la génération de réponse:**
En plus des modèles `tiers 2` ci dessus, envisager les modèles suivants, assez performants pour des questions de moindre complexité, et particulièrement peu chers;

* mistralai/mistral-small-3.2-24b-instruct: $0.06/M input tokens | $0.18/M output tokens
* meta-llama/llama-4-maverick: $0.15/M input tokens | $0.60/M output tokens
* z-ai/glm-4.5-air: $0.104/M input tokens | $0.68/M output tokens
* google/gemma-3-27b-it: $0.04/M input tokens | $0.15/M output tokens




## **Tableau de combinaisons (Grid Search)**

| Combinaison | Embedding | LLM (Création) | LLM (Génération) | Coût Relatif | Gestion de la complexité |
|-------------|-----------|----------------|------------------|--------------|------------|
| **C1 - Baseline Économique** | gemma emb | google/gemma-3-27b-it | google/gemma-3-27b-it | Très faible | Simple |
| **C2 - Hybride Léger** | gemma emb | deepseek-v3.1-terminus | mistral small | Faible | moyenne |
| **C3 - Optimal Qualité** | gemma emb | deepseek/deepseek-v3.2 | deepseek/deepseek-v3.2 | Moyen | Elevée |
| **C4 - Maximum Performance** | gemini ou openai large | openai/gpt-5.1 | openai/gpt-5.1 | Très élevé | Elevée|


## **Métriques d'Évaluation par combinaison**

### **Pour chaque combinaison, mesurer :**
1. **Qualité RAG**
   - Précision et couverture de la réponse VS référence
   - RAGAS (Faithfulness, Answer Relevancy)
   - Hallucinations

2. **Performance Technique**
   - Temps création graphe / génération réponses
   - Coût pour création de graphe
   - Coût par requête

3. **Qualité Graphe**
   - Nb de catégories formées
   - Nb de noeuds formés
   - Nombre de connexions



## **Matrice de Décision**

| Critère | Poids | C1 | C2 | C3 | C4 |
|---------|-------|----|----|----|----|
| **Qualité Réponse** | 70% | ⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| **Coût création graphe** | 10% | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ | ⭐ |
| **Coût génération réponse** | 15% | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐ |
| **Temps** | 5% | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐ |
| **Score Total** | 100% | **58** | **68** | **72** | **77** |

## **Recommandations**

1. **Démarrer avec C3** (Optimal Qualité)
* Si objectif qualité atteint (proche de C4 - Maximum Performance), tester C3 et mesurer la dégradation
* Si objectif qualité non atteint passer à C4

